In [2]:
from typing import List, Tuple, Dict, Any, Optional
from pandarallel import pandarallel
import os
import re
import sys
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [8]:
DIR_QASM_FILES = Path(
    "..", "program_bank", "v003", "2024_10_18__10_19__qiskit"
)

SAMPLE_SIZE = 10

TMP_DIR = Path(
    "..", ".tmp"
)
TMP_DIR.mkdir(exist_ok=True)

In [7]:
def get_all_qasm_files(folder: str) -> List[str]:
    return list(Path(folder).rglob("*.qasm"))


all_qasm_files = get_all_qasm_files(DIR_QASM_FILES)


sample = np.random.choice(all_qasm_files, SAMPLE_SIZE, replace=False)
for s in sample:
    print(s)

../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_3124_c9221e_qc_pytket.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_366_0c8c0f_qc_pennylane.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_1161_9bcff4_qc_pytket.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_579_0e9aee_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_3470_f4330f_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_753_09b667_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_2040_758cb0_qc_pennylane.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_1850_1943b2_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_149_253ea4_qc_qiskit.qasm
../program_bank/v003/2024_10_18__10_19__qiskit/qiskit_circuit_32q_10g_2234_bcf548_qc_pennylane.qasm


In [16]:
from bqskit import Circuit
from bqskit import compile as bcompile

# try to import all the circuits


def load_circuit(file: str) -> Optional[Circuit]:
    try:
        return Circuit.from_file(file)
    except Exception as e:
        print(f"Error loading {file}: {e}")
        return None


# get try to export all the circuits
def export_circuit(circuit: Circuit, file: str) -> None:
    try:
        circuit.save(file)
    except Exception as e:
        print(f"Error exporting {file}: {e}")
        return None


# load all the circuits / and export them in the current tmp director
circuits = [load_circuit(str(file)) for file in tqdm(sample)]
exported_files = [TMP_DIR / f"{i}.qasm" for i in range(SAMPLE_SIZE)]

for circuit, file in zip(circuits, exported_files):
    if circuit is not None:
        export_circuit(circuit, str(file))

  0%|          | 0/10 [00:00<?, ?it/s]

In [26]:
from bqskit.ext import qiskit_to_bqskit
from bqskit.ext import bqskit_to_qiskit

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

qreg = QuantumRegister(5)
creg = ClassicalRegister(5)
qc = QuantumCircuit(qreg, creg)
qc.h(0)
qc.cx(0, 1)
qc.measure_all(inplace=True)

print(qc.draw())

bqc = qiskit_to_bqskit(qc)

# export to qasm
bqc.save(str(TMP_DIR / "qiskit_to_bqskit.qasm"))


qc_back = bqskit_to_qiskit(bqc)
print(qc_back.draw())

        ┌───┐      ░ ┌─┐            
  q1_0: ┤ H ├──■───░─┤M├────────────
        └───┘┌─┴─┐ ░ └╥┘┌─┐         
  q1_1: ─────┤ X ├─░──╫─┤M├─────────
             └───┘ ░  ║ └╥┘┌─┐      
  q1_2: ───────────░──╫──╫─┤M├──────
                   ░  ║  ║ └╥┘┌─┐   
  q1_3: ───────────░──╫──╫──╫─┤M├───
                   ░  ║  ║  ║ └╥┘┌─┐
  q1_4: ───────────░──╫──╫──╫──╫─┤M├
                   ░  ║  ║  ║  ║ └╥┘
  c1: 5/══════════════╬══╬══╬══╬══╬═
                      ║  ║  ║  ║  ║ 
meas: 5/══════════════╩══╩══╩══╩══╩═
                      0  1  2  3  4 


QASM2ParseError: "<input>:6,5: 'c1' is already defined"

## Try to fix pennylane order

In [13]:
from pennylane.tape import QuantumTape
import pennylane as qml
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

qreg = QuantumRegister(5)
creg = ClassicalRegister(5)
qc = QuantumCircuit(qreg, creg)
qc.h(4)
qc.cx(3, 4)
qc.measure_all(inplace=True)


# Convert Qiskit circuit to a simplified form
simplified_qiskit_circ = qc.decompose().decompose()
n_qubits = simplified_qiskit_circ.num_qubits

# add at least one empty operation on each qubit to avoid the error
prefix_circ = QuantumCircuit(n_qubits)
for i in range(n_qubits):
    prefix_circ.id(i)
simplified_qiskit_circ = prefix_circ.compose(simplified_qiskit_circ)

# Define measurement and PennyLane device
measurements = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
circuit_fn = qml.from_qiskit(
    simplified_qiskit_circ, measurements=measurements)
dev = qml.device('default.qubit', wires=n_qubits)
qml_circuit = qml.QNode(circuit_fn, dev)

# Extract the QASM from the PennyLane QNode
with QuantumTape(shots=10) as tape:
    qml_circuit.construct([], {})
    qasm_str_pennylane = qml_circuit.tape.to_openqasm(wires=sorted(tape.wires))

print(qasm_str_pennylane)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
id q[0];
id q[1];
id q[2];
id q[3];
id q[4];
u3(1.5707963267948966,0.0,3.141592653589793) q[4];
cx q[3],q[4];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];



In [ ]:
from mqt import qcec
import tempfile
from qiskit.qasm2 import dumps

# Create a temporary file to store the QASM
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f1:
    f1.write(qasm_str_pennylane)
    f.flush()
    qasm_file_1 = f1.name

    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".qasm") as f2:
        f2.write(dumps(simplified_qiskit_circ))
        f2.flush()

        # Compute the QCEC
        qcec_val = qcec.verify(qasm_file_1, f2.name)